<a href="https://colab.research.google.com/github/BrightonOtieno/Classification-IrisDataSet-Tensorflow/blob/master/Classification_Iris_DataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%tensorflow_version 2.x

In [4]:
from __future__ import  absolute_import,division,print_function,unicode_literals

In [5]:
import tensorflow as tf
import pandas as pd

In [7]:
CSV_COLUMN_NAMES = ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES = ['Setosa','Versicolor','Virginica']

In [12]:
train_path = tf.keras.utils.get_file('iris_training.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')

In [13]:
test_path = tf.keras.utils.get_file('iris_testing.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

In [15]:
train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
test = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)


In [16]:
test.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


In [17]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [18]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [19]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [20]:
test_y.head()

0    1
1    2
2    0
3    1
4    1
Name: Species, dtype: int64

In [21]:
print(train_y.shape)
print(train.shape)

(120,)
(120, 4)


In [22]:
print(test_y.shape)
print(test.shape)

(30,)
(30, 4)


## Input Function

In [24]:
def input_fn(features,labels,training=True,batch_size=256):
  #convert the input into Dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
  # shuffle and repeat while Training
  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

## Feature Columns
* loop through the columns in  the train data frame
* confirm that columns are numeric

In [25]:
my_feature_columns= []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


# Building Classification Model
*  we could use
### LinearClassifier Model (except that it assumes linear relationship between  features and response/label)

### DNNClassifier works great with non-linear too 

In [26]:
classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns,
                                        hidden_units=[30,10],
                                        n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_4yfrdka', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
classifier.train(
    input_fn = lambda:input_fn(train,train_y,training=True),
    steps = 5000
)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp_4yfrdka/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into

# Model Evaluation

In [32]:
eval_result =  classifier.evaluate(input_fn=lambda:input_fn(test,test_y,training=False))
print('\nTest set accuracy:{accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-09-06T06:10:01Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp_4yfrdka/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.22754s
INFO:tensorflow:Finished evaluation at 2020-09-06-06:10:01
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.93333334, aver

# Predictions

In [ ]:
def input_fn(features,batch_size=256):
   # Takes in parameters such as (sepal lenght and ,width petal length and Width)
   #convert the inputs into Dataset without Labels
   return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

# Input Parameters
features = ['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict = {}
# Promt user to enter data
print('please type NUMERIC VALUES as Prompted')
for feature  in features:
  valid = True
  while valid:
    val = input(feature+ ":")
    if not  val.isdigit():
      valid = false # will break out of the loop

    else: # implying that the input value is a digit
      predict[feature] = [float(val)] 
      # conver value to float and pass it into the dictionary as a list(tensorflow style) 

# Do the predictions
predictions = classifier.predict(
    input_fn = lambda:input_fn(predict)
)

# Print the CLASS and PROBABILITY 
for prediction in predictions:
  class_id = prediction['class_ids'][0]
  probability = prediction['probabilities'][class_id]
  print('Prediction is "{}"({:.1f}%)'.format(SPECIES[class_id],100*probability))


